In [32]:
import pandas as pd
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier

In [33]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix,classification_report
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB, CategoricalNB
from sklearn.linear_model import LogisticRegression
from sklearn import model_selection
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.pipeline import Pipeline

### 5 mins change in stock price results (df2) (can ignore)

In [34]:
#for getting the price change (price before tweet and price after the tweet) #5mins interval one
df2 = pd.read_csv("tweets_stocks_combined_final_60mins.csv")

In [35]:
df2.shape

(2246, 18)

In [36]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2246 entries, 0 to 2245
Data columns (total 18 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   id                      2246 non-null   float64
 1   text                    2198 non-null   object 
 2   favorites               2246 non-null   int64  
 3   retweets                2246 non-null   int64  
 4   date                    2246 non-null   object 
 5   tweet_datetime          2246 non-null   object 
 6   date_part               2246 non-null   object 
 7   time_part               2246 non-null   object 
 8   hour                    2246 non-null   int64  
 9   year                    2246 non-null   int64  
 10  month                   2246 non-null   int64  
 11  topic                   2246 non-null   int64  
 12  compound                2246 non-null   float64
 13  datetime_60mins_after   2246 non-null   object 
 14  price_60mins_after      2246 non-null   

In [37]:
df2 = df2[df2['text'].notnull()]

In [38]:
#abs_diff has been calculated as price_5mins after - price now

In [39]:
min(df2.loc[:,"60mins_price_diff_abs"])

-3.989999999999952

In [40]:
max(df2.loc[:,"60mins_price_diff_abs"])

2.9699999999999704

In [41]:
min(df2.loc[:,"compound"])

-0.9736

In [42]:
df2

,id,text,favorites,retweets,date,tweet_datetime,date_part,time_part,hour,year,month,topic,compound,datetime_60mins_after,price_60mins_after,price_now,60mins_price_diff_abs,60mins_price_diff_perc
0,9.353400e+17,thank rand,42793,9125,2017-11-28 02:50:00,2017-11-28 10:50:00,2017-11-28,10:50:00,10,2017,11,1,0.4199,2017-11-28 11:50:00,261.485000,261.100000,0.385000,0.001475
1,8.997980e+17,join live fort myer arlington virginia,36009,4891,2017-08-22 01:00:00,2017-08-22 09:00:00,2017-08-22,09:00:00,9,2017,8,2,0.2960,2017-08-22 10:00:00,244.260000,243.670000,0.590000,0.002421
2,8.939700e+17,thank nicole,43367,8275,2017-05-08 23:01:00,2017-05-09 07:01:00,2017-05-09,07:01:00,7,2017,5,6,0.4199,2017-05-09 08:01:00,239.940000,239.875000,0.065000,0.000271
3,8.819770e+17,thank shawn steel nice word,50956,7465,2017-03-07 20:44:00,2017-03-08 04:44:00,2017-03-08,04:44:00,4,2017,3,3,0.6486,2017-03-08 05:44:00,237.022857,236.880000,0.142857,0.000603
4,8.787250e+17,make america great,134210,36346,2017-06-24 21:23:00,2017-06-25 05:23:00,2017-06-25,05:23:00,5,2017,6,2,0.6588,2017-06-25 06:23:00,243.326476,243.320762,0.005714,0.000023
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2241,9.990960e+17,person placed early campaign spy put previous ...,78529,20098,2018-05-23 01:13:00,2018-05-23 09:13:00,2018-05-23,09:13:00,9,2018,5,3,0.6792,2018-05-23 10:13:00,271.930000,271.040000,0.890000,0.003284
2242,9.874600e+17,general michael flynn life totally destroyed s...,93569,25259,2018-04-20 10:34:00,2018-04-20 18:34:00,2018-04-20,18:34:00,18,2018,4,3,-0.6689,2018-04-20 19:34:00,267.025000,266.820000,0.205000,0.000768
2243,9.870960e+17,thought prayer condolence family friend collea...,62645,16081,2018-04-19 22:30:00,2018-04-20 06:30:00,2018-04-20,06:30:00,6,2018,4,2,0.6900,2018-04-20 07:30:00,269.070000,268.620000,0.450000,0.001675
2244,9.863570e+17,today court decision mean congress must close ...,56749,12426,2018-04-17 21:34:00,2018-04-18 05:34:00,2018-04-18,05:34:00,5,2018,4,2,-0.9493,2018-04-18 06:34:00,270.695000,270.600000,0.095000,0.000351


In [43]:
#creates a list of values for the y label for each row
target_value=[]

for i in range(0, len(df2)):
    
    if df2.iloc[i, 16] > 0:
        target_value.append(1)
        #i+=1
    elif df2.iloc[i, 16] < 0:
        target_value.append(-1)
        #i+=1
    elif df2.iloc[i, 16] == 0:
        target_value.append(0)
        #i+=1

In [44]:
compound_group = []

for i in range(0, len(df2)):
    
    if df2.iloc[i, 12] >= 0.6:
        compound_group.append(15)
        
    elif df2.iloc[i, 12] >= 0.1:
        compound_group.append(14)
      
    elif df2.iloc[i, 12] > -0.1 and df2.iloc[i, 12] < 0.1:
        compound_group.append(13)
        
    elif df2.iloc[i, 12] <= -0.6:
        compound_group.append(11)
    
    elif df2.iloc[i, 12] <= -0.1:
        compound_group.append(12)

In [45]:
hours_grouped = []

for i in range(0, len(df2)):
    
    if df2.iloc[i, 8] >= 16:
        hours_grouped.append(20)
        
    elif df2.iloc[i, 8] >= 12:
        hours_grouped.append(19)
        
    else:
        hours_grouped.append(18)
      
        


In [46]:
#create a column for the y_label 
df2.loc[:, 'hours_grouped'] = hours_grouped
df2.loc[:,'target'] = target_value
df2.loc[:, 'compound_group'] = compound_group

In [47]:
#to check if correctly classified
#df2[df2['y_label']==-1].sum()

In [48]:
counter1 = 0
counter2 = 0
counter3 = 0
counter4 = 0
counter5 = 0


for i in range(0,len(df2)):
    
    if df2.iloc[i, 12] >= 0.6:
        counter1+=1
        
    elif df2.iloc[i, 12] >= 0.1:
        counter2+=1
      
    elif df2.iloc[i, 12] > -0.1 and df2.iloc[i, 12] < 0.1:
        counter3+=1
        
    elif df2.iloc[i, 12] <= -0.6:
        counter4+=1
    
    elif df2.iloc[i, 12] <= -0.1:
        counter5+=1
    

print(counter1, counter2, counter3, counter4, counter5) 

964 458 235 297 244


In [49]:
#checking if correctly classified
counter1 = 0
counter2 = 0
counter3 = 0
for i in range(0,len(df2)):
    if (df2.iloc[i, 16 ] > 0) == True:
        counter1+=1
        
    elif (df2.iloc[i, 16] < 0) == True:
        counter2+=1
    
    elif (df2.iloc[i, 16] == 0) == True:
        counter3+=1
    

print(counter1, counter2, counter3) 

1224 953 21


In [50]:
#there were 51 rows for which tweets were empty so those rows have been dropped

In [51]:
#to verify that there are no NA values
df2.isna().sum()

id                        0
text                      0
favorites                 0
retweets                  0
date                      0
tweet_datetime            0
date_part                 0
time_part                 0
hour                      0
year                      0
month                     0
topic                     0
compound                  0
datetime_60mins_after     0
price_60mins_after        0
price_now                 0
60mins_price_diff_abs     0
60mins_price_diff_perc    0
hours_grouped             0
target                    0
compound_group            0
dtype: int64

In [52]:
X = df2[['topic', 'compound_group', 'hours_grouped']]
y = df2.target

smote = SMOTE(random_state=0, sampling_strategy='all')
X_resampled, y_resampled = smote.fit_sample(X, y)

df2 = pd.DataFrame(X_resampled, columns=X.columns)

In [53]:
comp = pd.get_dummies(df2['compound_group'])
topic = pd.get_dummies(df2['topic'])
hrs = pd.get_dummies(df2['hours_grouped'])
# Drop column as it is now encoded
df2 = df2.drop(['compound_group'], axis = 1)
df2 = df2.drop(['topic'], axis = 1)
df2 = df2.drop(['hours_grouped'], axis = 1)
# Join the encoded df
df2 = df2.join(comp)
df2 = df2.join(topic)
df2 = df2.join(hrs)
df2

,11,12,13,14,15,0,1,2,3,4,5,6,7,8,9,18,19,20
0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0
1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0
2,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0
3,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0
4,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3667,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0
3668,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0
3669,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0
3670,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0


In [54]:
X = df2.iloc[:, :]
y = y_resampled
print(X.columns)
print(y.value_counts())

Int64Index([11, 12, 13, 14, 15, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 18, 19, 20], dtype='int64')
 1    1224
-1    1224
 0    1224
Name: target, dtype: int64


In [55]:
df = X.join(y)
df

,11,12,13,14,15,0,1,2,3,4,5,6,7,8,9,18,19,20,target
0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1
1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1
2,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1
3,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,1
4,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3667,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0
3668,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0
3669,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0
3670,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0


In [56]:
df.to_csv('ANN_model_all.csv', index=False)

In [57]:
# X = df2.iloc[:, 18:]
# y = df2.target
# print(X.columns)
# print(y.value_counts())

In [58]:
#Train-test split
#test size is 20% of the dataset
x_train,x_test,y_train,y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [59]:
clf = CategoricalNB()
model = clf.fit(x_train, y_train)

y_pred = model.predict(x_test)

print("Accuracy of Naive Bayes Classifier: {}%".format(round(accuracy_score(y_test, y_pred)*100,2)))
print("\nConfusion Matrix of Naive Bayes Classifier:\n")
print(confusion_matrix(y_test, y_pred))
print("\nClassification Report of Naive Bayes Classifier:\n")
print(classification_report(y_test, y_pred))

Accuracy of Naive Bayes Classifier: 45.85%

Confusion Matrix of Naive Bayes Classifier:

[[ 62  98  90]
 [ 21 170  35]
 [ 60  94 105]]

Classification Report of Naive Bayes Classifier:

              precision    recall  f1-score   support

          -1       0.43      0.25      0.32       250
           0       0.47      0.75      0.58       226
           1       0.46      0.41      0.43       259

    accuracy                           0.46       735
   macro avg       0.45      0.47      0.44       735
weighted avg       0.45      0.46      0.44       735



In [60]:
logr = LogisticRegression()
model = logr.fit(x_train, y_train)

y_pred = model.predict(x_test)

print("Accuracy of Logistic Regression Classifier: {}%".format(round(accuracy_score(y_test, y_pred)*100,2)))
print("\nConfusion Matrix of Logistic Regression Classifier:\n")
print(confusion_matrix(y_test, y_pred))
print("\nClassification Report of Logistic Regression Classifier:\n")
print(classification_report(y_test, y_pred))

Accuracy of Logistic Regression Classifier: 48.57%

Confusion Matrix of Logistic Regression Classifier:

[[ 70  94  86]
 [ 22 183  21]
 [ 65  90 104]]

Classification Report of Logistic Regression Classifier:

              precision    recall  f1-score   support

          -1       0.45      0.28      0.34       250
           0       0.50      0.81      0.62       226
           1       0.49      0.40      0.44       259

    accuracy                           0.49       735
   macro avg       0.48      0.50      0.47       735
weighted avg       0.48      0.49      0.46       735



In [61]:
clf = LinearSVC()
model = clf.fit(x_train, y_train)

y_pred = model.predict(x_test)

print("Accuracy of SVC Classifier: {}%".format(round(accuracy_score(y_test, y_pred)*100,2)))
print("\nConfusion Matrix of SVC Classifier:\n")
print(confusion_matrix(y_test, y_pred))
print("\nClassification Report of SVC Classifier:\n")
print(classification_report(y_test, y_pred))

Accuracy of SVC Classifier: 48.57%

Confusion Matrix of SVC Classifier:

[[ 67  97  86]
 [ 22 186  18]
 [ 60  95 104]]

Classification Report of SVC Classifier:

              precision    recall  f1-score   support

          -1       0.45      0.27      0.34       250
           0       0.49      0.82      0.62       226
           1       0.50      0.40      0.45       259

    accuracy                           0.49       735
   macro avg       0.48      0.50      0.47       735
weighted avg       0.48      0.49      0.46       735



In [62]:
clf = RandomForestClassifier()
model = clf.fit(x_train, y_train)

y_pred = model.predict(x_test)

print("Accuracy of Random Forest Classifier: {}%".format(round(accuracy_score(y_test, y_pred)*100,2)))
print("\nConfusion Matrix of Random Forest Classifier:\n")
print(confusion_matrix(y_test, y_pred))
print("\nClassification Report of Random Forest Classifier:\n")
print(classification_report(y_test, y_pred))

Accuracy of Random Forest Classifier: 52.52%

Confusion Matrix of Random Forest Classifier:

[[106  68  76]
 [ 37 177  12]
 [ 90  66 103]]

Classification Report of Random Forest Classifier:

              precision    recall  f1-score   support

          -1       0.45      0.42      0.44       250
           0       0.57      0.78      0.66       226
           1       0.54      0.40      0.46       259

    accuracy                           0.53       735
   macro avg       0.52      0.53      0.52       735
weighted avg       0.52      0.53      0.51       735

